In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

 Load the dataset

In [2]:
df = pd.read_csv('../data/data.csv')

1. Create Aggregate Features


We will group transactions by customer and compute aggregate statistics

In [3]:
# Aggregate features
aggregate_df = df.groupby('CustomerId').agg(
    total_transaction_amount=('Amount', 'sum'),
    avg_transaction_amount=('Amount', 'mean'),
    transaction_count=('TransactionId', 'count'),
    std_transaction_amount=('Amount', 'std')
).reset_index()

print(aggregate_df.head())

        CustomerId  total_transaction_amount  avg_transaction_amount  \
0     CustomerId_1                  -10000.0           -10000.000000   
1    CustomerId_10                  -10000.0           -10000.000000   
2  CustomerId_1001                   20000.0             4000.000000   
3  CustomerId_1002                    4225.0              384.090909   
4  CustomerId_1003                   20000.0             3333.333333   

   transaction_count  std_transaction_amount  
0                  1                     NaN  
1                  1                     NaN  
2                  5             6558.963333  
3                 11              560.498966  
4                  6             6030.478146  


2. Extract Time-based Features

In [4]:
# Convert 'TransactionStartTime' to datetime if it's not in that format
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])

# Extract hour, day, month, and year
df['transaction_hour'] = df['TransactionStartTime'].dt.hour
df['transaction_day'] = df['TransactionStartTime'].dt.day
df['transaction_month'] = df['TransactionStartTime'].dt.month
df['transaction_year'] = df['TransactionStartTime'].dt.year

print(df[['transaction_hour', 'transaction_day', 'transaction_month', 'transaction_year']].head())


   transaction_hour  transaction_day  transaction_month  transaction_year
0                 2               15                 11              2018
1                 2               15                 11              2018
2                 2               15                 11              2018
3                 3               15                 11              2018
4                 3               15                 11              2018
